<a href="https://colab.research.google.com/github/FXtest2023/FXtest2023/blob/main/1115_EURGBP_News_maxdepth5_v3_random_save1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas_ta
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218907 sha256=10b142bdbf2fafd05aaac612b7f9a00f36e364c49572c616bcbceb499da711b1
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 9.7 MB/s eta 0:00:00


In [2]:

from google.colab import files
import pandas as pd
import matplotlib.pyplot as plt
import pandas_ta as pd_ta
from sklearn.model_selection import KFold
from optuna.integration import lightgbm as lgb
#import lightgbm as lgb
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
import numpy as np
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split
import random

In [3]:
currencypair = 'EURGBP'
market1 = 'EZ'
currency1 = 'EUR'
market2 = 'GB'
currency2 = 'GBP'

In [4]:
url = "https://raw.githubusercontent.com/FXtest2023/FXtest2023/main/" + currencypair + "_1D.csv"
df_price = pd.read_csv(url)

In [5]:
df_price['diff']  = df_price['bid_close'].shift(-1) - df_price['bid_close']
df_price['label'] = df_price['diff'].apply(lambda x: 1 if x>0 else 0 )
df_price.dropna(inplace=True)
df_price.reset_index(drop=True,inplace=True)

In [6]:
news_url1 = "https://raw.githubusercontent.com/FXtest2023/FXtest2023/main/" + market1 + "_market_news_1D.csv"
df_news1 =  pd.read_csv(news_url1)

In [7]:
news_url2 = "https://raw.githubusercontent.com/FXtest2023/FXtest2023/main/" + market2 + "_market_news_1D.csv"
df_news2 =  pd.read_csv(news_url2)

In [8]:
currency_news_url1 = "https://raw.githubusercontent.com/FXtest2023/FXtest2023/main/" + currency1 + "_currency_news_1D.csv"
df_currency_news1 =  pd.read_csv(currency_news_url1)

In [9]:
currency_news_url2 = "https://raw.githubusercontent.com/FXtest2023/FXtest2023/main/" + currency2 + "_currency_news_1D.csv"
df_currency_news2 =  pd.read_csv(currency_news_url2)

In [10]:
df_1H = df_price.merge(df_news1, how='inner', on='utc_datetime')
df_1H = df_1H.merge(df_news2, how='inner', on='utc_datetime')

df_1H = df_1H.merge(df_currency_news1, how='inner', on='utc_datetime')
df_1H = df_1H.merge(df_currency_news2, how='inner', on='utc_datetime')


In [11]:
df_1H = df_1H.loc[df_1H.utc_datetime >'2019'][df_1H.utc_datetime <'2023-11']
df_1H

,utc_datetime,bid_open,bid_high,bid_low,bid_close,diff,label,stockindexbuzz_x,bondbuzz_x,ratesbuzz_x,...,emotionvsfact_y,marketrisk_y,longshort_y,longshortfrcst_y,pricedirection_y,priceforecast_y,volatility_y,carrytrade_y,curpeginstablty_y,pricemomentum_y
0,2019-01-02 00:00:00+00:00,0.89931,0.89996,0.89833,0.89861,0.00520,1,530.0,124.5,116.0,...,0.2450,0.0311,-0.0053,0.0000,0.0238,0.0026,0.0291,0.0,0.0000,0.0079
1,2019-01-03 00:00:00+00:00,0.89861,0.91103,0.89806,0.90381,-0.00188,0,1658.0,1397.1,937.5,...,0.3233,0.0036,0.0004,0.0025,-0.0775,-0.0036,0.0313,0.0,0.0000,-0.0004
2,2019-01-04 00:00:00+00:00,0.90381,0.90615,0.90093,0.90193,-0.00700,0,1274.6,935.7,619.0,...,0.2717,0.0095,-0.0111,-0.0016,-0.0559,-0.0010,0.0281,0.0,0.0000,0.0016
3,2019-01-05 00:00:00+00:00,0.90193,0.90279,0.89433,0.89493,0.00096,1,1363.3,568.5,608.0,...,0.2314,0.0302,-0.0043,-0.0005,-0.0065,0.0031,0.0241,0.0,0.0000,-0.0092
4,2019-01-07 00:00:00+00:00,0.89527,0.89637,0.89521,0.89589,0.00188,1,65.5,240.2,158.5,...,0.2768,0.0156,0.0057,0.0133,-0.0028,-0.0161,0.0434,0.0,0.0000,0.0057
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1503,2023-10-26 00:00:00+00:00,0.87100,0.87323,0.87030,0.87295,-0.00264,0,2664.7,1279.4,2121.5,...,0.1963,0.0279,-0.0077,0.0010,0.0040,0.0066,0.0256,0.0,0.0000,0.0000
1504,2023-10-27 00:00:00+00:00,0.87296,0.87332,0.86900,0.87031,0.00169,1,3441.2,2089.0,4710.5,...,0.2185,0.0046,-0.0236,-0.0070,-0.0390,-0.0039,0.0293,0.0,0.0000,0.0003
1505,2023-10-28 00:00:00+00:00,0.87034,0.87255,0.86942,0.87200,-0.00017,0,2839.4,1161.4,1907.0,...,0.2520,0.0132,-0.0196,-0.0029,-0.0054,-0.0046,0.0231,0.0,0.0000,-0.0045
1506,2023-10-30 00:00:00+00:00,0.86933,0.87192,0.86933,0.87183,0.00061,1,127.9,332.0,431.0,...,0.2407,0.0052,-0.0183,-0.0022,-0.0016,-0.0158,0.0199,0.0,-0.0006,-0.0028


In [12]:

print(list(df_1H.columns))

['utc_datetime', 'bid_open', 'bid_high', 'bid_low', 'bid_close', 'diff', 'label', 'stockindexbuzz_x', 'bondbuzz_x', 'ratesbuzz_x', 'stkidxsentiment_x', 'stkidxoptimism_x', 'sidxtrust_x', 'sidxfear_x', 'sidxstress_x', 'sidxsurprise_x', 'sidxuncertainty_x', 'sidxmarketrisk_x', 'sidxprcdirection_x', 'sidxpricefrcst_x', 'sidxvolatility_x', 'bsentiment_x', 'boptimism_x', 'btrust_x', 'bfear_x', 'bstress_x', 'bsurprise_x', 'buncertainty_x', 'bdefault_x', 'bprcdirection_x', 'bpricefrcst_x', 'bvolatility_x', 'centralbank_x', 'debtdefault_x', 'interestrates_x', 'intratesfrcst_x', 'mplcyloosevstight_x', 'stockindexbuzz_y', 'bondbuzz_y', 'ratesbuzz_y', 'stkidxsentiment_y', 'stkidxoptimism_y', 'sidxtrust_y', 'sidxfear_y', 'sidxstress_y', 'sidxsurprise_y', 'sidxuncertainty_y', 'sidxmarketrisk_y', 'sidxprcdirection_y', 'sidxpricefrcst_y', 'sidxvolatility_y', 'bsentiment_y', 'boptimism_y', 'btrust_y', 'bfear_y', 'bstress_y', 'bsurprise_y', 'buncertainty_y', 'bdefault_y', 'bprcdirection_y', 'bpricefrcs

In [13]:
# 乱数シードの固定
#SEED = 3407

# LightGBM訓練用関数を定義
def trainGbdt(X_train, X_val, y_train, y_val):

    #LGB用のデータに変形
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_val = lgb.Dataset(X_val, y_val)

# need to set deterministic and force_col_wise
# https://github.com/microsoft/LightGBM/issues/3761
    param = {
            'objective': 'binary',
            'metric': 'auc',
            'boosting_type': 'gbdt',
            # 'random_seed':SEED,
            # 'deterministic': True,
            # 'force_col_wise': True,
            'max_depth':5,
            'verbose':-1,
#            'min_data_in_leaf': 60
#            'device': 'gpu',
 #           'force_col_wise': True,
        }

        # 訓練実施
    model = lgb.train(param,
                    lgb_train,
                    valid_sets=[lgb_train,lgb_val],
                    valid_names=('train', 'val'),
#                    callbacks=([lgb.early_stopping(stopping_rounds=100)]),
#                    verbose_eval=False
                    )
    print(model.params, model.best_iteration, model.best_score)

    return model

In [14]:
df_1H_cols = list(df_1H.columns)
no_feature_cols = ['utc_datetime','bid_open',	'bid_high',	'bid_low','bid_close', 'diff','label']
stockindex_cols = ['sidxfear_x','sidxtrust_x','sidxsurprise_x','sidxvolatility_x','sidxstress_y','stkidxoptimism_y']
bond_cols = ['bpricefrcst_x','bsentiment_x','bsurprise_y','boptimism_y']
rates_cols = ['interestrates_x','mplcyloosevstight_x','interestrates_y','intratesfrcst_y','debtdefault_y']
currency_cols = ['priceforecast_x','pricemomentum_x','conflict_x','lovehate_y','conflict_y','surprise_y','timeurgency_y']
high_corr_cols = ['sidxfear_x','priceforecast_x','interestrates_y','lovehate_y']

result = pd.DataFrame(columns = ['feature','number of features', 'train acc','validation acc','test acc','F score','random seed', 'best_feature','features'])
test_cols = {
             "stockindex": stockindex_cols,
             "bond": bond_cols,
             "rates": rates_cols,
             "currency": currency_cols,
             "high_corr": high_corr_cols,
             "stockindex_bond": stockindex_cols + bond_cols,
             "stockindex_rates": stockindex_cols + rates_cols,
             "stockindex_currency": stockindex_cols + currency_cols,
             "bond_rates": bond_cols + rates_cols,
             "bond_currency": bond_cols + currency_cols,
             "rates_currency": rates_cols + currency_cols,
             "stockindex_bond_rates": stockindex_cols + bond_cols + rates_cols,
             "stockindex_bond_currency": stockindex_cols + bond_cols + currency_cols,
             "stockindex_rates_currency": stockindex_cols + rates_cols + currency_cols,
             "bond_rates_currency": bond_cols + rates_cols + currency_cols,
             "stockindex_bond_rates_currency": stockindex_cols + bond_cols + rates_cols + currency_cols

            }
for i in range(12):
    for k, v in test_cols.items():
        feature_cols = v
        cols = no_feature_cols + feature_cols

        df = df_1H[cols]
        df_train = df[ df['utc_datetime']<'2022-07']
        df_val = df[(df['utc_datetime']>'2022-07') & (df['utc_datetime']<'2023')]

        df_test = df[(df['utc_datetime']>'2023') & (df['utc_datetime']<'2023-09')]
        X_train=df_train.drop(columns=no_feature_cols)
        y_train=df_train['label']

        X_val=df_val.drop(columns=no_feature_cols)
        y_val=df_val['label']


        X_test=df_test.drop(columns=no_feature_cols)
        y_test=df_test['label']

        model_result = trainGbdt(X_train, X_val, y_train, y_val)

        y_pred_train = model_result.predict(X_train, num_iteration=model_result.best_iteration)
        y_pred_val = model_result.predict(X_val, num_iteration=model_result.best_iteration)
        y_pred_test = model_result.predict(X_test, num_iteration=model_result.best_iteration)

        new_row = {'feature': k,
               'number of features': len(v),
               'train acc': accuracy_score(y_train, np.round(y_pred_train)),
               'validation acc': accuracy_score(y_val, np.round(y_pred_val)),
               'test acc': accuracy_score(y_test, np.round(y_pred_test)),
               'F score': f1_score(y_test, np.round(y_pred_test)),
               'random seed': "rand",
               'best_feature': feature_cols[np.argmax(model_result.feature_importance(importance_type='gain'))],
               'features': feature_cols}
        result.loc[len(result)] = new_row

result = result.sort_values(['feature', 'test acc'], ascending = [True, False])

result.to_csv(currencypair +'_news_1D_test_v3.csv', index=False, encoding = 'utf-8-sig')
files.download(currencypair+'_news_1D_test_v3.csv')



[I 2023-11-15 11:09:06,885] A new study created in memory with name: no-name-2735604b-27d4-423e-9b98-26fc38a8df63
min_child_samples, val_score: 0.612565: 100%|##########| 5/5 [00:01<00:00,  3.20it/s]
[I 2023-11-15 11:10:09,778] A new study created in memory with name: no-name-fac82a61-26a9-4bcc-8da4-cd3da580a8ff


{'objective': 'binary', 'metric': 'auc', 'boosting_type': 'gbdt', 'max_depth': 5, 'verbose': -1, 'feature_pre_filter': False, 'lambda_l1': 3.241712143926372, 'lambda_l2': 9.112296435374239, 'num_leaves': 31, 'feature_fraction': 1.0, 'bagging_fraction': 1.0, 'bagging_freq': 0, 'min_child_samples': 10, 'num_iterations': 1000} 0 defaultdict(<class 'collections.OrderedDict'>, {'train': OrderedDict([('auc', 0.9475285605739684)]), 'val': OrderedDict([('auc', 0.6125645994832042)])})


min_child_samples, val_score: 0.522610: 100%|##########| 5/5 [00:06<00:00,  1.20s/it]
[I 2023-11-15 11:10:56,767] A new study created in memory with name: no-name-fd40385e-03b1-4323-88f0-58c556b2de4d


{'objective': 'binary', 'metric': 'auc', 'boosting_type': 'gbdt', 'max_depth': 5, 'verbose': -1, 'feature_pre_filter': False, 'lambda_l1': 6.50126752606843e-05, 'lambda_l2': 1.744815974525592e-08, 'num_leaves': 11, 'feature_fraction': 0.7, 'bagging_fraction': 0.40219718932404075, 'bagging_freq': 7, 'min_child_samples': 20, 'num_iterations': 1000} 0 defaultdict(<class 'collections.OrderedDict'>, {'train': OrderedDict([('auc', 0.9903072236903153)]), 'val': OrderedDict([('auc', 0.522609819121447)])})


min_child_samples, val_score: 0.491279: 100%|##########| 5/5 [00:01<00:00,  2.79it/s]
[I 2023-11-15 11:11:30,010] A new study created in memory with name: no-name-986c8669-2c4f-418d-80b1-da424bf2519e


...

{'objective': 'binary', 'metric': 'auc', 'boosting_type': 'gbdt', 'max_depth': 5, 'verbose': -1, 'feature_pre_filter': False, 'lambda_l1': 0.0, 'lambda_l2': 0.0, 'num_leaves': 2, 'feature_fraction': 0.4, 'bagging_fraction': 0.816806466041034, 'bagging_freq': 6, 'min_child_samples': 5, 'num_iterations': 1000} 0 defaultdict(<class 'collections.OrderedDict'>, {'train': OrderedDict([('auc', 0.8240002958599776)]), 'val': OrderedDict([('auc', 0.48094315245478036)])})


min_child_samples, val_score: 0.476744: 100%|##########| 5/5 [00:08<00:00,  1.67s/it]

{'objective': 'binary', 'metric': 'auc', 'boosting_type': 'gbdt', 'max_depth': 5, 'verbose': -1, 'feature_pre_filter': False, 'lambda_l1': 0.0, 'lambda_l2': 0.0, 'num_leaves': 17, 'feature_fraction': 0.6, 'bagging_fraction': 1.0, 'bagging_freq': 0, 'min_child_samples': 20, 'num_iterations': 1000} 0 defaultdict(<class 'collections.OrderedDict'>, {'train': OrderedDict([('auc', 1.0)]), 'val': OrderedDict([('auc', 0.47674418604651164)])})


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>